In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
from scipy.optimize import minimize

In [2]:
def daily_returns(stock):
    stock = stock.copy()
    
    stock['Returns'] = 0.0
    
    for i in range(1, len(stock)):
        previous_close = stock.loc[stock.index[i-1], 'Close']
        current_close = stock.loc[stock.index[i], 'Close']
        
        stock.loc[stock.index[i], 'Returns'] = (current_close - previous_close) / previous_close

    stock['Returns'] = stock['Returns'].fillna(0)

    return stock


In [3]:
# Expected return of the stock
def expected_returns(stock):
    stock = daily_returns(stock)
    stock_mean = sum(stock['Returns'])/len(stock)
    return stock_mean
        

In [4]:
# Standard deviation of stock
def std(stock):
    stock = daily_returns(stock)
    stand_dev = np.std(stock['Returns'])
    return stand_dev

In [5]:
# Variance of stock
def var(stock):
    variance = std(stock) ** 2
    return variance

In [6]:
# Covariance of 2 stocks
def cov(stock_a, stock_b):
    
    if len(stock_a) != len(stock_b):
        raise ValueError("Both stocks must have the same number of price points.")
    else:
        returns_a = daily_returns(stock_a)['Returns']
        returns_b = daily_returns(stock_b)['Returns']
    
        returns_data = np.vstack([returns_a, returns_b])
        cov_ab = np.cov(returns_data)[0][1]  
    return cov_ab


In [7]:
def historical_data(stock):
    print("Expected return is", expected_returns(stock))
    print("Standard deviation is", std(stock))

In [8]:
# weights
weights = []
for w1 in range(1,101):
    for w2 in range(1,101-w1):
        w3 = 100 - w1 - w2
        if w3 != 0:
            weights.append([w1,w2,w3])
                
weight = np.array(weights)
portfolio = pd.DataFrame(weights, columns = ['Weight_a', 'Weight_b', 'Weight_c'] )

In [9]:
def port(stock_a, stock_b, stock_c):
    # returns
    sr_a = expected_returns(stock_a)
    sr_b = expected_returns(stock_b)
    sr_c = expected_returns(stock_c)
    sr = np.array([sr_a, sr_b, sr_c])

    # Variance
    vr_a = var(stock_a)
    vr_b = var(stock_b)
    vr_c = var(stock_c)

    # Covariance
    cv_ab = cov(stock_a, stock_b)
    cv_bc = cov(stock_b, stock_c)
    cv_ca = cov(stock_c, stock_a)

    # return           
    l = len(portfolio)
    for i in range (0,l):
        wg = np.array([portfolio.loc[portfolio.index[i], 'Weight_a'], portfolio.loc[portfolio.index[i], 'Weight_b'], portfolio.loc[portfolio.index[i], 'Weight_c']] )
        portfolio.loc[portfolio.index[i], 'Port_mean'] = np.sum(wg * sr*100)

   
    # Portfolio Variance
    l = len(portfolio)
    for i in range (0,l):

        # weights
        wg_a = portfolio.loc[portfolio.index[i], 'Weight_a']
        wg_b = portfolio.loc[portfolio.index[i], 'Weight_b']
        wg_c = portfolio.loc[portfolio.index[i], 'Weight_c']

        # Calculation partwise
        part_1 = (wg_a ** 2) * (vr_a)
        part_2 = (wg_b ** 2) * (vr_b)
        part_3 = (wg_c ** 2) * (vr_c)
        part_4 = 2 * wg_a * wg_b * cv_ab
        part_5 = 2 * wg_b * wg_c * cv_bc
        part_6 = 2 * wg_c * wg_a * cv_ca

        # Final calculation
        portfolio.loc[portfolio.index[i], 'Port_var'] = part_1 + part_2 + part_3 + part_4 + part_5 + part_6


    # Sharpe's ratio
    for i in range (0,l):
        portfolio.loc[portfolio.index[i], "Sharpe_ratio"] = (portfolio.loc[portfolio.index[i], "Port_mean"] - rt)/(portfolio.loc[portfolio.index[i], "Port_var"]**0.5)


    return portfolio.sort_values(by = 'Sharpe_ratio', ascending= False)

In [68]:
def max_min(stock_a, stock_b, stock_c):
    p = port(stock_a, stock_b, stock_c)
    max_return = p['Port_mean'].max()
    min_return = p['Port_mean'].min()
    max_var = p['Port_var'].max()
    min_var = p['Port_var'].min()


    print("Out of", len(p), "Portfolios:")
    print("Maximum return:", max_return)
    print("Minimum return:", min_return)
    print("Maximum variance:", max_var)
    print("Minimum variance:", min_var)

    p.sort_values(by = 'Sharpe_ratio', ascending= False)
    stock_a_weight = p.loc[p.index[0],'Weight_a']
    stock_b_weight = p.loc[p.index[0],'Weight_b']
    stock_c_weight = p.loc[p.index[0],'Weight_c']
    port_return = p.loc[p.index[0],'Port_mean']
    port_var = p.loc[p.index[0],'Port_var']
    sharpe_rt = p.loc[p.index[0],'Sharpe_ratio']

    
    print("Here are the optimal stock weights after optimizing the portfolio for maximum Sharpe ratio.")
    print(list[0], "Weights =", stock_a_weight)
    print(list[1], "Weights =", stock_b_weight)
    print(list[2], "Weights =", stock_c_weight)
    print("Maximum return =", port_return)
    print("Minimum risk with the return of", port_return, "is", port_var)
    print("Optimal Sharpe's ratio is", sharpe_rt)


    print("Are you satisfied? Y/N: ")
    print("Y for Yes")
    print("N for No")
    a = input().upper()

    if a == "Y":
        print("Thank you!!! Hope your portfolio grows so much, it needs its own personal assistant!")

    elif a == "N":
        while True:
            
            print("What's your preference — are you looking for your expected return or the level of risk (variance)?: ")
            print("R for Return")
            print("V for Risk(Variance)")
            z = input().upper()
        
            # Return
            if z == "R":
                while True:
                    rt = float(input("How much return would you like to earn?: "))
            
                    if (rt <= max_return) & (rt>= min_return):
                        condition = (p['Port_mean'] >= rt) & (p['Port_mean'] < rt + 1)
                        if condition.any():
                            pf = p.loc[condition].sort_values(by = ['Port_mean','Sharpe_ratio'], ascending=False)[:10]
                            pf.columns = [ticker_1, ticker_2, ticker_3, "Return", "Risk", "Sharpe's ratio"]
                            print(pf)
                            print("Are you satisfied? Y/N: ")
                            a = input().upper()
                            if a == "Y": 
                                print("Thank you!!! Hope your portfolio grows so much, it needs its own personal assistant!")
                                break

                            elif a =="N":
                                continue

                            else:
                                print("Invalid input")
                                
    
                    else:
                        print("This is stepping outside your portfolio’s comfort zone—like picking a pizza topping that doesn’t belong!")


                break
                
            # Risk
            elif z == "V":
                while True:
                    rt = float(input("How much risk would you like to bear?: "))
        
                    if (rt <= max_var) & (rt>= min_var):
                        condition = (p['Port_var'] >= rt) & (p['Port_var'] < rt + 1)
                        if condition.any():
                            df = p.loc[condition].sort_values(by = ['Port_var'])
                            pf = df.sort_values(by = ['Sharpe_ratio'], ascending=False)[:10]
                            pf.columns = [ticker_1, ticker_2, ticker_3, "Return", "Risk", "Sharpe's ratio"]
                            print(pf)
                            print("Are you satisfied? Y/N: ")
                            a = input().upper()
                            if a == "Y": 
                                print("Thank you!!! Hope your portfolio grows so much, it needs its own personal assistant!")
                                break

                            elif a == "N":
                                continue

                            else:
                                print("Invalid input")
                                
    
                    else:
                        print("This is stepping outside your portfolio’s comfort zone—like picking a pizza topping that doesn’t belong!")


                break
    
                
    
    else:
        print("Invalid input")

In [70]:
ticker_1 = input("Enter 1st Ticker's Id:").strip().upper()
ticker_2 = input("Enter 2nd Ticker's Id:").strip().upper()
ticker_3 = input("Enter 3rd Ticker's Id:").strip().upper()

list = []
list.append(ticker_1)
list.append(ticker_2)
list.append(ticker_3)


print("Provide the date range in the format (yyyy-mm-dd)")
start_date = input("Enter Start date:").strip()
end_date = input("Enter End date:").strip()
rt = float(input("Enter risk free rate:"))
        
try:
    start_date = datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = datetime.strptime(end_date, "%Y-%m-%d").date()
    
except ValueError:
    print("Error: Please enter valid dates in the format yyyy-mm-dd.")
        
    
stock_a = yf.download(ticker_1, start=start_date, end=end_date)
stock_b = yf.download(ticker_2, start=start_date, end=end_date)
stock_c = yf.download(ticker_3, start=start_date, end=end_date)

stock_a.columns = [col[0] for col in stock_a.columns]
stock_b.columns = [col[0] for col in stock_b.columns]
stock_c.columns = [col[0] for col in stock_c.columns]
    
    
max_min(stock_a, stock_b, stock_c)


Enter 1st Ticker's Id: mahmf
Enter 2nd Ticker's Id: aapl
Enter 3rd Ticker's Id: msft


Provide the date range in the format (yyyy-mm-dd)


Enter Start date: 2021-01-23
Enter End date: 2025-01-23
Enter risk free rate: 7.365


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Out of 4851 Portfolios:
Maximum return: 31.90258678844724
Minimum return: 6.366774570209234
Maximum variance: 55.774513709024035
Minimum variance: 2.2142421469700118
Here are the optimal stock weights after optimizing the portfolio for maximum Sharpe ratio.
MAHMF Weights = 58
AAPL Weights = 1
MSFT Weights = 41
Maximum return = 22.261834596999567
Minimum risk with the return of 22.261834596999567 is 20.10020919300318
Optimal Sharpe's ratio is 3.322719705287248
Are you satisfied? Y/N: 
Y for Yes
N for No


...

      MAHMF  AAPL  MSFT     Return       Risk  Sharpe's ratio
3773     53     4    43  20.990030  16.972894        3.307193
3830     54    15    31  20.986441  17.503996        3.255776
3886     55    26    19  20.982853  18.093098        3.201490
3941     56    37     7  20.979265  18.740200        3.144902
3774     53     5    42  20.967793  16.965029        3.302560
3831     54    16    30  20.964205  17.500169        3.250816
3887     55    27    18  20.960616  18.093309        3.196244
3942     56    38     6  20.957028  18.744448        3.139409
3775     53     6    41  20.945556  16.957519        3.297892
3832     54    17    29  20.941968  17.496697        3.245822
Are you satisfied? Y/N: 


 n
How much return would you like to earn?:  30


      MAHMF  AAPL  MSFT     Return       Risk  Sharpe's ratio
4836     94     1     5  30.938512  51.338159        3.290062
4837     94     2     4  30.916275  51.334892        3.287063
4838     94     3     3  30.894038  51.331978        3.284053
4839     94     4     2  30.871801  51.329420        3.281031
4840     94     5     1  30.849564  51.327215        3.277998
4830     93     1     6  30.697493  50.259242        3.291192
4831     93     2     5  30.675256  50.255836        3.288166
4832     93     3     4  30.653019  50.252785        3.285129
4833     93     4     3  30.630782  50.250088        3.282081
4834     93     5     2  30.608545  50.247745        3.279020
Are you satisfied? Y/N: 


 y


Thank you!!! Hope your portfolio grows so much, it needs its own personal assistant!


In [12]:
# MAHMF = 1004
# AAPl = 1004
# MSFT = 1004
# BHARTIARTL.NS = 987
# HCLTECH.NS = 987